In [ ]:
#----------------------------------------------------
# Code for plot color-magnitude diagrams F335M vs. F300M-F335M 
#
#              Jimena Rodriguez
#                    2024
#---------------------------------------------------
# Import Functions
#---------------------------------------------------


from astropy.io import ascii
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from astropy.convolution import Gaussian2DKernel, convolve
from scipy.stats import gaussian_kde

#----------------------------------------------------
#List of PHANGS-JWST Cycle 1 galaxies
#----------------------------------------------------

galaxy_names=['ngc5068','ic5332','ngc628','ngc3351','ngc3627','ngc2835','ngc4254','ngc4321','ngc4535','ngc1087','ngc4303',
          'ngc1385','ngc1566','ngc1433','ngc7496','ngc1512','ngc1300','ngc1672','ngc1365']

#----------------------------------------------------
# Read the files with the 3 sigma and 5 sigma color dispersion curve parameters
#----------------------------------------------------
fit_table3=ascii.read('..../ComplementaryData/Coef_dispersion_color_curve0_3sigma.csv') # your path to this files
fit_table5=ascii.read('.../ComplementaryData/Coef_dispersion_color_curve0_5sigma.csv')  # your path to this files

#----------------------------------------------------
# Read file with the F300M and F335M 5 sigma detection limit 
# These limits were computed using randomly positioned apertures, as explained in Rodriguez+25 
#----------------------------------------------------
mag_lim=ascii.read('..../ComplementaryData/Table_335_300_200_five_sigma.csv') # your path to this files


figure = plt.figure(figsize=(20, 20))
figure.patch.set_facecolor('white')
x=0.01
y=0.21

#Apertures Correction in F335M and F300M
# used in Rodriguez+25
#----------------------------------------------------
ac_F335M=-0.66
ac_F300M=-0.68


# density percentiles that want to plot
#----------------------------------------------------
levels = [0.05,0.1, 0.3, 0.5, 0.7, 0.9]
for galaxy in galaxy_names:
    
    if galaxy=='ngc628':
        galaxy_p='ngc0628'
    else:
        galaxy_p=galaxy
        
    #----------------------------------------------------    
    # Define all parameters used to delimit the selection  
    # of 3.3µm PAH emitters.  
    #----------------------------------------------------
    F335_lim=mag_lim['F335M_ab_4p'][mag_lim['galaxy']==galaxy]+ac_F335M 
    F300_lim=mag_lim['F300M_ab_4p'][mag_lim['galaxy']==galaxy]+ac_F300M
    a3=fit_table3['coef_a'][fit_table3['galaxy']==galaxy]
    b3=fit_table3['coef_b'][fit_table3['galaxy']==galaxy]
    c3=fit_table3['coef_c'][fit_table3['galaxy']==galaxy]
    a5=fit_table5['coef_a'][fit_table5['galaxy']==galaxy]
    b5=fit_table5['coef_b'][fit_table5['galaxy']==galaxy]
    c5=fit_table5['coef_c'][fit_table5['galaxy']==galaxy]
    
    
    if galaxy=='ngc5068':
        magnitudes=np.linspace(20,30,20)
    else:
        magnitudes=np.linspace(20,28,20)
        
    
    #----------------------------------------------------
    # 'table' file: Contains the photometry of all sources detected in F335M.  
    # 'pop2_lim' file: Contains 3.3µm PAH emitters detected at 3–5σ,  
    #                  following the methodology explained in Rodriguez+25.  
    # 'pop3_lim' file: Contains 3.3µm PAH emitters detected at >5σ,  
    #                  following the methodology explained in Rodriguez+25.  
    #----------------------------------------------------
    table=ascii.read('..../ComplementaryData//Phot_v5p4_src_335_'+galaxy+'.csv') # your path to this files
    
    pop2_lim=ascii.read('..../ComplementaryData/population2_'+galaxy+'.csv') # your path to this files
    pop3_lim=ascii.read('..../ComplementaryData/population3_'+galaxy+'.csv')  # your path to this files

    
#----------------------------------------------------    
# Density contours  
#----------------------------------------------------
    
    xm=table["F300M_ab"]-table["F335M_ab"]
    ym=table["F335M_ab"]
    xp=xm[~np.isnan(xm) & ~np.isnan(ym) ]
    yp=ym[~np.isnan(xm) & ~np.isnan(ym) ]
    data = np.vstack([xp, yp])
    
    #estimate the probability density function of the data using a Gaussian kernel
    #----------------------------------------------------
    kde = gaussian_kde(data)

    # Evaluate the density at each point
    #----------------------------------------------------
    density = kde(data)

    # Determine the density threshold for the 5th percentile
    #----------------------------------------------------
    threshold = np.percentile(density, 5)

    # Identify points that are outside the 5th percentile contour
    #----------------------------------------------------
    outside_contour = density < threshold

    # Create and plot density contours
    #----------------------------------------------------
    ax= figure.add_axes([x, y, 0.11, 0.14])
    sns.kdeplot(x=xm, y=ym, ax=ax, levels=levels, color='dimgrey', legend=False)
    

#----------------------------------------------------    
# PLOT  
#----------------------------------------------------
    
        
    ax.scatter(xp[outside_contour], yp[outside_contour], marker=',', alpha=0.1, c='dimgrey', s=1) # points outside the contours
    ax.scatter(pop2_lim['F300M_ab']-pop2_lim['F335M_ab'],pop2_lim['F335M_ab'], marker='.', alpha=0.5, c='b', s=4) #3-5sigma PAH emitters
    ax.scatter(pop3_lim['F300M_ab']-pop3_lim['F335M_ab'],pop3_lim['F335M_ab'], marker='.', alpha=0.5, c='g', s=4) #>5 sigma PAH emitters
    xx=np.linspace(18,F335_lim,10)  
    ax.plot((a3 * np.exp(b3 * magnitudes) + c3),magnitudes,  c='red', ls='-', lw=2, label='+3$\sigma_{curve}$') #3 sigma curve
    ax.plot(-(a3 * np.exp(b3 * magnitudes) + c3), magnitudes, c='red', ls='--', lw=2, label='-3$\sigma_{curve}$') #-3 sigma curve
    ax.plot((a5 * np.exp(b5 * magnitudes) + c5),magnitudes,  c='darkorange', ls='-', lw=2, label='+5$\sigma_{curve}$') #5 sigma curve
    ax.axhline(y=F335_lim, c='dodgerblue',ls='--', lw=2, label='F335M detection limit')  #F335M detection limit 
    ax.plot(F300_lim-xx,xx, lw=2, c='deeppink', ls='--', label='F300M detection limit')  #F300M detection Limit
    ax.axvline(x=0, c='teal', lw=2, ls='-')
    
    ax.tick_params(axis='x', direction='in', length=6)
    ax.tick_params(axis='y', direction='in', length=6) 
    ax.set_ylim(26,16.5)
    ax.set_xlim(-2.,6) 
    yticks = ax.get_yticks()
    ytick_labels = ax.get_yticklabels()
    ytick_labels[-1]='' 
    ax.set_xticklabels(ax.get_xticklabels(), fontsize=18)
    ax.set_yticklabels(ax.get_yticklabels(), fontsize=18)
    ax.set_title(galaxy.upper(), y=0.8,loc='right',  backgroundcolor= 'silver', alpha=0.7, fontsize=20)
    if(x<=0.01):        
        ax.set_ylabel('F335M', size=20)
    if (y>-0.2): 
            ax.set_xticklabels([])
    if ((y<-0.2) | (galaxy=='ngc7496') ):    
        ax.set_xlabel('F300M-F335M', size=20)
    if x > 0.01:
        ax.set_yticklabels([])
        ax.set_ylabel('')
    if x > 0.01 and y > -0.3:
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_ylabel('')
    if (x>=0.33): 
        x=0.01
        y=y-0.14
    else:
        x=x+0.11
    
    if galaxy=='ngc1365':
        ax.legend(bbox_to_anchor=(1.05,0.7), loc='upper left', fontsize=18)
        
        
# Save plot 
#----------------------------------------------------
plt.savefig('.../CMD_color_dispersion_curve_countors_limits.png', transparent=False, bbox_inches='tight') # path to your output folder